# Introduction to Ray
---
(*Suggested Time To Complete: 30 minutes*)

Welcome, we're glad to have you along! This module serves as an interactive introduction to Ray, a flexible distributed computing framework built for Python with data science and machine learning practitioners in mind. Before we jump into the structure of this tutorial, let us first unpack the context of where we are coming from, along with the motivation for learning Ray.

![Main Map](../_static/assets/Introduction_to_Ray/map.png)

*Figure 1*

**Context**

Today's artificial intelligence (AI) applications require enormous amounts of data to be trained on and machine learning (ML) models tend to grow over time. From consumer-facing products like recommendation systems and photo editing software to enterprise-level use cases like reducing downtime in manufacturing and order-fulfillment optimization, ML systems have become so complex and infrastructure intensive that developers have no option but to distribute execution across multiple machines. However, distributed computing is hard. It requires specialized knowledge about orchestrating clusters of computers together to efficiently schedule tasks and must provide features like fault tolerance when a component fails, high availability to minimize service interruption, and autoscaling to reduce waste.

As a data scientist, machine learning pracitioner, developer, or engineer, your contribution may center on building data processing pipelines, training complicated models, running efficient hyperparameter experiments, creating simulations of agents, and/or serving your application to users. In each case, you need to choose a distributed system to support each task, but you don't want to learn a different programming language or toss out your existing toolbox. This is where Ray comes in.

**What is Ray?**

Ray is an open source, distributed execution framework that allows you to scale AI and machine learning workloads. Our goal is to keep things simple (enabled by a concise core API) so that you can parallelize Python programs on your laptop, cluster, cloud, or even on-premise with minimal code changes. Ray automatically handles all aspects of distributed execution including orchestration, scheduling, fault tolerance, and auto scaling so that you can scale your apps without becoming a distributed systems expert. With a rich ecosystem of libraries and integrations with many important data science tools, Ray lowers the effort needed to scale compute intensive workloads.

**Notebook Outline**

This first notebook is part of a series where we will discuss the three major **layers** that comprise Ray, namely its core engine, high-level libraries, and ecosystem of integrations. In this first notebook, we will cover:

- Introduction to Ray
- Part One: Ray Core
    - Ray Core Key Concepts
    - Example: Cross-Validation on Housing Data
        - Sequential Implementation
        - Distributed Implementation with Ray
    - Summary
- Homework
- Next Steps

**Prerequisites**

To gain the most from this notebook, it helps if you have a working knowledge of Python as well as previous experience with machine learning. The ideal learner has minimal familiarity with Ray and is interested in leveraging Ray's simple distributed computing framework to scale AI and Python workloads.

**Learning Goals**

Upon completion of this module, you will have a intuition for:
1. An Overview of Ray
2. Ray Core Key Concepts
2. Ray Core Key API Elements
3. How to Navigate Next Steps to Start Scaling Workloads

# Part One: Ray Core
---

![Ray Core](../_static/assets/Introduction_to_Ray/Ray_Core.png)

*Figure 2*

Ray Core is a low-level, distributed computing framework for Python with a concise core API, and you can think of it as the foundation that Ray's data science libraries (Ray AIR) and third-party integrations (Ray Ecosystem) are built on. This simple and general-purpose Python library enables every developer to easily build scalable, distributed systems that run on your laptop, cluster, cloud or Kubernetes.

A key strength lies in Ray Core's simple primitives: Tasks, Actors, and Objects.

**Tasks:** Ray enables you to designate functions to be executed asychronously on separate Python workers. These asynchronous Ray functions, *tasks*, can specify their resource requirements in terms of CPUs, GPUs, and custom resources which are used by the cluster scheduler to distribute tasks for parallelized execution.

**Actors:** What tasks are to functions, actors are to classes. An actor is a stateful worker and methods of the actor are scheduled on that specific worker and can access and mutate the state of that worker.

**Objects:** In Ray, tasks and actors create and compute on objects, and we refer to these objects as *remote objects* because they can be stored anywhere in a Ray cluster. We use *object references* to refer to them, and they are cached in Ray's distributed shared memory *object store*.

Ray sets up and manages clusters of computers so that you can run distributed tasks on them. For now, the best way to understand Ray Core is to motivate it with some popular use cases and then try our hand at parallelizing a program outselves.

**Most Popular Use Cases**

- Parallel Model Selection
- Parameter Servers
- Distributing Reinforcement Learning Training

### Coding Excercise: Housing Prices with `sklearn`
***
Let's take a look at how we can apply Ray Core's flexible and simple API to scale a bare bones version of a common ML task: cross-validation.

Here, we have a dataset of [California Housing](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html) with 20,640 samples and features including `[longitude, latitude, housing_median_age, total_rooms, total_bedrooms, population, households, median_income, median_house_value, ocean_proximity]`. Given that we want to use a linear regression model, we want to assess how the results will generalize to an unseen independent dataset, say, on new housing data coming in this year. To do this, we would try cross-validation which is a model validation technique that resamples different portions of the data to train and test a model on different iterations. After we conduct these trials, we can average the error to get an estimate of the model's predictive performance.

However, training the same model multiple times on different subsets of a dataset can take a long time, especially if you're working with a much more complex model and larger dataset. Pictured below in Figure 3, the sequential approach trains each model one after another in a series.

![Sequential Timeline](../_static/assets/Introduction_to_Ray/Sequential_Timeline.png)

*Figure 3*

In this example, we will first implement the sequential approach, then improve it by distributing training with Ray Core, and finally compare the code differences to highlight how minimal the changes are.

#### Import Relevant Packages

In [ ]:
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

num_trials = 100
X, y = fetch_california_housing(return_X_y=True, as_frame=True)

#### Train 100 Models Sequentially
Here, we will define a function that randomly splits our housing dataset into testing and training subsets (in the style of Monte Carlo Cross-Validation, where subsets are generated without replacement and have non-unique subsets from round to round). `sequential()` then fits a model, generates predictions, and returns the R-squared score (closer to 1 = better performance, closer to 0 = worse performance).

Then, we'll train 100 models on these random splits, one after another, and finally print out the average of the rounds.

In [ ]:
%%time

def sequential():
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    model = LinearRegression()

    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    r2 = metrics.r2_score(y_test, predictions)

    return r2

errors_seq = []

for i in range (num_trials):
    errors_seq.append(sequential())

print(sum(errors_seq) / num_trials)


#### From Sequential -> Parallel

We just trained 100 linear regression models in a series and averaged their R-squared values in about ~1 second. Let's now leverage Ray to train these models in parallel (where multiple tasks may happen simultaneously) and see a runtime improvement. In Figure 4, you can visually inspect the difference where the scheduler assigns each available worker (in this timeline, we chose `n = 4` workers) a task. The scheduler itself has a nontrivial overhead involved with communicating between workers and other cluster management.

![Distributed_Timeline](../_static/assets/Introduction_to_Ray/Distributed_Timeline.png)

*Figure 4*

With just a few code changes, we will modify our existing Python program to distribute it among *n* number of workers. Of course, this is a lightweight example, but it's illustrative of the kind of user experience you get with Ray Core's lean API.

Notice in Figure 5 that we need to use four API calls:

1. `ray.init()` - initialize a Ray context
2. `@ray.remote` - a decorator that turns functions into tasks and classes into actors
3. `.remote()` - postfix to every remote function, remote class declaration, or invocation of a remote class method; returns an `ObjectID` associated with the work to be done
4. `ray.get()` - returns an object or list of objects from the object ID

You may notice that instead of storing the result of `train.remote()` directly into a list of `errors`, we instead store it in a list called `obj_ids`. Once you run a Ray remote function, it will immediately return an `ObjectID` (or 'Object Reference'). This `ObjectID` is a *promise* of future work, meaning that the task is delegated to a worker, an `ObjectID` is returned while the task executes in the background, and in order to access the expected output, you need to call `ray.get()` on the `ObjectID`

![Housing Diff](../_static/assets/Introduction_to_Ray/Housing_Diff.png)

*Figure 5*

And with just a few lines of difference, we're able to parallelize training without having to concern ourselves with orchestration, fault tolerance, autoscaling, or anything else that requires specialized knowledge of distributed systems.

#### Train 100 Models in Parallel with Ray
To start, we'll import Ray (check out our [installation instructions](https://docs.ray.io/en/latest/ray-overview/installation.html)) and start a Ray cluster on our local machine that can utilize all the cores available on your computer as workers. We use `ray.is_initialized` to allow us to make sure that we only have one Ray cluster active.

In [ ]:
import ray

if ray.is_initialized:
    ray.shutdown()

ray.init()

As illustrated above in Figure 5, we will:
1. Add the decorator `@ray.remote` to our function `distributed()` to specify that it is a task to be run remotely. 
2. Then we call that function as `distributed.remote()` in the `for` loop to append to our list of object ids. 
3. Finally, we fetch the result outside of the loop to access the final error list (as to not *block* the launching of remote tasks asynchronously) and print out the average R-squared value.

In [ ]:
%%time

@ray.remote
def distributed():
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    model = LinearRegression()

    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    r2 = metrics.r2_score(y_test, predictions)

    return r2

obj_ids = []

for i in range (num_trials):
    obj_ids.append(distributed.remote())

errors_dist = ray.get(obj_ids)

print(sum(errors_dist) / num_trials)

And now you've done it! You have distributed the training of 100 models in a very through round of cross-validation on our California Housing dataset. Compare the runtime for each method of training 100 models. Is this what you expected?

### Optional Excercise: Quick Sort 
***
If you would like to test your knowledge of the Ray Core concepts we covered in the housing example, try your hand at building a distributed version of the classic algorithm, quick sort.

At a glance, quick sort selects an element from the array as a pivot, partitions the array into two sub-arrays (according to whether they are less than or greater than the pivot), then sorts them recursively. You can preview an animation of quick sort in action below in Figure 6.

Quick sort is an example of a divide and conquer algorithm, a strategy of solving a large problem by recursively breaking the problem down into smaller sub-problems. By nature, this strategy lends itself well to a parallel implementation because the execution of each operation independently solves smaller instances of the same problem which is then merged into the final solution.

Let's walkthrough the mechanics a little deeper in the code sample that follows.

![Quick Sort](https://upload.wikimedia.org/wikipedia/commons/6/6a/Sorting_quicksort_anim.gif)

*Figure 6*

#### Sequential Implementation of Quick Sort
Here is the classic version that you may be familiar with below.

1. Given an array, `partition` uses the last element as the pivot and creates two subarrays of elements less than the pivot and elements greater than the pivot (`greater` will always be empty in this case).
2. A call to `quick_sort_sequential` partitions the array into two subarrays and the pivot element, then calls itself by passing in the two newly created subarrays, and finally merges the sorted answer.
3. We use the `if len(collection) <= 500000` condition because it will be useful later in the distributed version to avoid tiny tasks. We'll discuss it further in subsequent tutorials, but if you are curious, find more information [here](https://docs.ray.io/en/latest/ray-core/tips-for-first-time.html#tip-2-avoid-tiny-tasks).

Notice that each merge depends on the recursive call below it to return before completing its task.

In [ ]:
def partition(collection):
    pivot = collection.pop()
    greater, lesser = [], []
    for element in collection:
        if element > pivot:
            greater.append(element)
        else:
            lesser.append(element)
    return lesser, pivot, greater

def quick_sort_sequential(collection):
    if len(collection) <= 500000:
        return sorted(collection)
    lesser, pivot, greater = partition(collection)
    lesser = quick_sort_sequential(lesser)
    greater = quick_sort_sequential(greater)
    return lesser + [pivot] + greater

#### Your Turn: Distributed Implementation of Quick Sort

In the space below, try to apply the key concepts in the section before to create a distributed implementation of quick sort. We'll start you out with some base of code to modify, but it's up to you finish it!

In [ ]:
import ray
if ray.is_initialized:
    ray.shutdown()

### MODIFY THIS CODE BELOW ###
def quick_sort_distributed(collection):
    if len(collection) <= 500000:
        return sorted(collection)
    lesser, pivot, greater = partition(collection)
    lesser = quick_sort_sequential(lesser)
    greater = quick_sort_sequential(greater)
    return lesser + [pivot] + greater

#### Compare Both Versions on a Large Array

Let's now compare these two implementations by timing each and inspecting the results. We'll create a large random array and time both methods to see how long each takes. Note that we use `ray.put()` to store the array in the object store as to not pass the same large object as an argument repeatedly.

In [ ]:
import time
from numpy import random

size = 10_000_000

unsorted = random.randint(1000000, size=(size)).tolist()

s = time.time()
quick_sort_sequential(unsorted)
print(f"Sequential Duration: {(time.time() - s):.3f}")

s = time.time()
unsorted_obj = ray.put(unsorted)
ray.get(quick_sort_distributed.remote(unsorted_obj))
print(f"Distributed Duration: {(time.time() - s):.3f}")

### Summary
1. Introduced to Ray Core and Most Popular Workloads
2. Key Concepts of Ray Core
3. Sequential -> Distributed Training of 100 Models

#### [Key Concepts](https://docs.ray.io/en/latest/ray-core/key-concepts.html)
- Tasks
- Actors
- Objects

#### [Key API Elements in This Section](https://docs.ray.io/en/latest/ray-core/package-ref.html#python-api)
- `ray.init()`
- `@ray.remote`
- `.remote()`
- `ray.get()`
- `ray.put()`

#### Next
Now that we've covered the core engine, let's go up one layer of abstraction to look at a suite of data science libraries build on top of Ray Core to target specific machine learning workloads in the next notebook!

# Homework
---
If you would like to practice your new skills further with some in-depth examples beyond the embedded coding excercises, take a look at this list of suggested problems:
- [Read About Debugging and Profiling on Ray](https://docs.ray.io/en/latest/ray-core/troubleshooting.html)
    - Dig into how to observe Ray work by visualizing tasks in the Ray timeline, profiling using Python's CProfile, understanding crashes and suboptimal performance, and more in this user guide.
- [Distribute a Classical Algorithm with Ray](https://github.com/ray-project/hackathon5-algo)
    - In this excercise, go to the GitHub repo linked above for details on choosing a classic algorithm implemented in Python, editing the implementation to parallelize the work with Ray, and compare your results against the sequential implementation.


# Next Steps
---
Congratulations! You have completed your first tutorial on an Introduction to Ray and Ray Core! We introduced the three layers of Ray: Core, AIR, and the Ecosystem. In this notebook, we explored Ray Core's key concepts of tasks, actors, and objects along with key API elements through examples. In the next module, we will talk about Ray AI Runtime, a set of native libraries built on top of Ray Core specialized for machine learning workloads.

From here, you can learn and get more involved with our active community of developers and researchers by checking out the following resources:
- [Ray's "Getting Started" Guides](https://docs.ray.io/en/latest/ray-overview/index.html): A collection of QuickStart Guides for every library including installation walkthrough, examples, blogs, talks, and more!
- [Official Ray Website](https://www.ray.io/): Browse the ecosystem and use this site as a hub to get the information that you need to get going and building with Ray.
- [Join the Community on Slack](https://forms.gle/9TSdDYUgxYs8SA9e8): Find friends to discuss your new learnings in our Slack space.
- [Use the Discussion Board](https://discuss.ray.io/): Ask questions, follow topics, and view announcements on this community forum.
- [Join a Meetup Group](https://www.meetup.com/Bay-Area-Ray-Meetup/): Tune in on meet-ups to listen to compelling talks, get to know other users, and meet the team behind Ray.
- [Open an Issue](https://github.com/ray-project/ray/issues/new/choose): Ray is constantly evolving to improve developer experience. Submit feature requests, bug-reports, and get help via GitHub issues.